In [ ]:
import pandas as pd

new_oura = pd.read_csv("C:/Users/rocke/Downloads/oura_sleep.csv")  
test_col = new_oura["bedtime_start"].str[:10]  # taking in the date without time
hour_col = new_oura["bedtime_start"].str[11:13]  # extracting the time
test_col_date = pd.to_datetime(test_col)  # converting to datetime, will be useful later
test_col_date
new_oura["bedtime_start"] = test_col_date  # just creating columns for easier subsetting
new_oura["bedtime_hour"] = hour_col
new_oura.rename(columns={"bedtime_start":"date"}, inplace=True)  # renaming for convenience
new_oura

In [ ]:
# creating a column that tells you if the participant slept past midnight for that day, and if they started
# to sleep during the day, I will drop them. note that times are listed in military time (ie. 0:00 - 24:00)
past_midnight = []
for i in new_oura["bedtime_hour"]:
    if int(i) < 5:
        past_midnight.append(True)
    elif 5 <= int(i) <= 18:
        past_midnight.append("Drop")
    else:
        past_midnight.append(False)
new_oura["past_midnight"] = past_midnight

# admittedly, this won't account for people who nap before 5 am. (low chance, but possible. especially within a population of healthcare workers)

In [ ]:
new_dates = []
import datetime as datetime
for i in range(len(new_oura["bedtime_hour"])):
    if new_oura["past_midnight"][i] == True:
        # substract 1 if they slept past midnight, consistent with our method of lagging sleep by 1 day
        new_day = new_oura["date"][i] - datetime.timedelta(days=1) 
        new_dates.append(new_day)
    else:
        # change nothing to the date if they did not sleep past midnight
        new_dates.append(new_oura["date"][i])
new_dates

In [ ]:
new_oura["date"] = new_dates
new_oura = new_oura[new_oura["past_midnight"] != "Drop"]  # drop rows that had the value "Drop" in the past midnight thing

In [ ]:
# getting rid of the weird sleepers (slept ONE time between 5 am and 6 pm, or slept for multiple days)
sleep_duplicates = new_oura.duplicated(subset = ["participant_id", "date"], keep = "last")  # keep the LAST recorded sleep
new_oura["sleep_duplicates"] = sleep_duplicates
new_oura = new_oura[new_oura["sleep_duplicates"] == False]  # chooses the last row

In [ ]:
# checking to see if there are any duplicate dates :)
new_oura[["participant_id", "date"]].value_counts()

In [ ]:
new_stress = pd.read_csv("C:/Users/rocke/Downloads/daily_stress_measure(2).csv")
stress_takedate = new_stress["daily858_startdate"].str[:10]  # taking the date only
stress_convertdate = pd.to_datetime(stress_takedate)  # converting to datetime, useful later for when we merge
new_stress["daily858_startdate"] = stress_convertdate
new_stress.rename(columns={"daily858_startdate":"date"}, inplace=True)  # renaming
new_stress

In [ ]:
duplicated = new_stress.duplicated(subset = ["participant_id", "date"], keep = "last")  # taking only the last recorded survey response
new_stress["duplicated"] = duplicated
new_stress = new_stress[new_stress["duplicated"] == False]
new_stress

In [ ]:
# shifting each sleep variable DOWN by 1 day for EACH participant. Why? Naively shifting the whole data set means we are assuming we are only working with one participant
new_oura[['awake', 'bedtime_end', 'bedtime_end_delta', 'date',
       'bedtime_start_delta', 'breath_average', 'deep', 'duration',
       'efficiency', 'hr_5min', 'hr_average', 'hr_lowest', 'hypnogram_5min',
       'is_longest', 'light', 'midpoint_at_delta', 'midpoint_time',
       'onset_latency', 'period_id', 'rem', 'restless', 'rmssd', 'rmssd_5min',
       'score', 'score_alignment', 'score_deep', 'score_disturbances',
       'score_efficiency', 'score_latency', 'score_rem', 'score_total',
       'summary_date', 'temperature_delta', 'temperature_deviation',
       'temperature_trend_deviation', 'total']] = new_oura.groupby(['participant_id'])['awake', 'bedtime_end', 'bedtime_end_delta', 'date',
       'bedtime_start_delta', 'breath_average', 'deep', 'duration',
       'efficiency', 'hr_5min', 'hr_average', 'hr_lowest', 'hypnogram_5min',
       'is_longest', 'light', 'midpoint_at_delta', 'midpoint_time',
       'onset_latency', 'period_id', 'rem', 'restless', 'rmssd', 'rmssd_5min',
       'score', 'score_alignment', 'score_deep', 'score_disturbances',
       'score_efficiency', 'score_latency', 'score_rem', 'score_total',
       'summary_date', 'temperature_delta', 'temperature_deviation',
       'temperature_trend_deviation', 'total'].shift(1)

In [ ]:
new_oura = new_oura[new_oura["score"].notnull()]  # taking away the nan's produced by shifting 
new_oura

In [ ]:
merged_tables = pd.merge(new_oura, new_stress, how = "left", on = "participant_id")  # left merge on participant id

In [ ]:
final_merge_table = merged_tables[merged_tables["date_x"] == merged_tables["date_y"]]  # then selecting by dates

# here, we managed to have each observation unique by its date and participant id
final_merge_table

In [ ]:
# since there are 3 columns for working with covid, I made this binary. Shouldn't matter much if someone worked with covid
# earlier or later in their shift (assumption)
work_with_covid = [] 
for i in range(len(final_merge_table["daily_covid_shifts___1"])):
    if (final_merge_table["daily_covid_shifts___1"].iloc[i] == 0) and (final_merge_table["daily_covid_shifts___2"].iloc[i] == 0):
        work_with_covid.append(0)
    else:
        work_with_covid.append(1)
final_merge_table["worked_with_covid"] = work_with_covid
final_merge_table

In [ ]:
# making sure there are no duplicates after the merge
final_merge_table[["participant_id", "date_x"]].value_counts()

In [ ]:
# merging ptsd survey data with our data set
ptsd = pd.read_csv("C:/Users/rocke/Downloads/ptsd_checklist_pclc.csv")
ptsd_merge = pd.merge(final_merge_table, ptsd, how = "inner", on = "participant_id")
ptsd_merge

In [ ]:
# reading in the ace survey data set
ace_survey = pd.read_csv("C:/Users/rocke/Downloads/adverse_childhood_events_ace.csv")

In [ ]:
# now mering ace with our updated data set as well
ace_ptsd_merge = pd.merge(ptsd_merge, ace_survey, how = "inner", on = "participant_id")
ace_ptsd_merge

In [ ]:
# reading in the demographics data set
dem = pd.read_csv("C:/Users/rocke/Downloads/demographics_survey(1).csv")

In [ ]:
# finally, merging demographic survey data to our updated data set
oura1lag_dem = pd.merge(ace_ptsd_merge, dem, how = "left", on = "participant_id")

In [ ]:
# beautiful.
oura1lag_dem

In [ ]:
# saving final data set as a csv for easy use/access
oura1lag_dem.to_csv("C:/Users/rocke/Downloads/updated_oura_stress_ptsd_ace_1lag.csv")